In [21]:
import import_hack
import steps
from data_store import sqlite_table_datastore as sqlds
from data_store import file_system_directory_datastore as fsds
from data_store import stream_ndarray_adapter_datastore as snads
from data_store import numpy_datastore as npds
from transformer import transformers as trs
from search import inverted_multi_index_searcher as imis
from quantization import pq_quantizer
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
img_dir_path = r'C:\data\images\brodatz\data.brodatz\size_213x213'
sqlite_db_path='example_find_nearest_by_histograms_with_multi_index'
sqlite_table_name='histograms'
n_subquantizers=2
n_clusters=256
sqltie_clusters_table_name='clusters_pq_{0}_{1}'.format(n_subquantizers, n_clusters)

In [24]:
image_bytes_ds = fsds.FileSystemDirectoryDataStore(dir_path=img_dir_path)
histograms_ds=sqlds.SQLiteTableDataStore(sqlite_db_path, sqlite_table_name)
image_bytes_ds.get_count()

999

In [25]:
transformers_=[trs.BytesToNdarray(), trs.NdarrayToOpencvMatrix(), trs.OpencvMatrixToHistogram(graycolor=True)]
steps.transform_step(image_bytes_ds, transformers_, histograms_ds)

In [26]:
histograms_ds_ndarrayds = snads.StreamNdarrayAdapterDataStore(histograms_ds)
print(histograms_ds_ndarrayds.get_items_sorted_by_ids().shape)
np_neighbor_ids_ds=npds.NumpyDataStore()

(999, 256, 1)


In [27]:
x=histograms_ds_ndarrayds.get_items_sorted_by_ids()
x_ids=histograms_ds_ndarrayds.get_ids_sorted()

In [30]:
pq_=pq_quantizer.PQQuantizer(n_clusters, n_subquantizers)
#be careful with pq: multiindex will consume at least (n_clusters^m_subquantizers)*4 bytes
cluster_centers_ds=sqlds.SQLiteTableDataStore(sqlite_db_path, sqltie_clusters_table_name)
steps.quantize_step(histograms_ds_ndarrayds, pq_, cluster_centers_ds)

In [31]:
cluster_centers_ndarray_ds=snads.StreamNdarrayAdapterDataStore(cluster_centers_ds)
cluster_centers=cluster_centers_ndarray_ds.get_items_sorted_by_ids()

In [32]:
cluster_centers.shape

(2, 256, 128)

In [33]:
searcher_=imis.InvertedMultiIndexSearcher(x, x_ids, cluster_centers)

In [34]:
np_neighbor_ids_ds=npds.NumpyDataStore()
steps.search_step(histograms_ds, searcher_, 20 , np_neighbor_ids_ds)

In [35]:
np_neighbor_ids_ds.get_items_sorted_by_ids()

array([[  1,   3,   8, ..., 754, 555, 842],
       [  9,   2,   6, ...,  99,  98,  97],
       [  3,   1,   9, ..., 555, 755, 754],
       ..., 
       [998, 997, 995, ..., 992,  45,  44],
       [998, 997, 995, ..., 476, 475, 648],
       [999, 992, 996, ..., 558, 556, 505]])

In [36]:
ids=histograms_ds_ndarrayds.get_ids_sorted()
source_ids=image_bytes_ds.get_ids_sorted()
np_neighbor_source_ids=npds.NumpyDataStore()
steps.transform_step(np_neighbor_ids_ds, [trs.TranslateByKeysTransformer(ids, source_ids)], np_neighbor_source_ids)

In [41]:
np_neighbor_source_ids.get_items_sorted_by_ids()

array([['D001_01.png', 'D001_03.png', 'D001_08.png', ..., 'D085_07.png',
        'D063_06.png', 'D095_05.png'],
       ['D001_09.png', 'D001_02.png', 'D001_06.png', ..., 'D011_09.png',
        'D011_08.png', 'D011_07.png'],
       ['D001_03.png', 'D001_01.png', 'D001_09.png', ..., 'D063_06.png',
        'D085_08.png', 'D085_07.png'],
       ..., 
       ['D112_08.png', 'D112_07.png', 'D112_05.png', ..., 'D112_02.png',
        'D005_09.png', 'D005_08.png'],
       ['D112_08.png', 'D112_07.png', 'D112_05.png', ..., 'D054_08.png',
        'D054_07.png', 'D073_09.png'],
       ['D112_09.png', 'D112_02.png', 'D112_06.png', ..., 'D063_09.png',
        'D063_07.png', 'D058_01.png']], dtype=object)

In [ ]:
import os
os.remove(sqlite_db_path)